## '제74주년 경찰의 날 기념식 축사' 전문을 읽고 TF-IDF를 계산하세요.

* 1) DataFrame을 생성

In [19]:
import os
import pyspark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [23]:
from pyspark.sql.types import StructType, StructField, StringType

_myrdd = spark.sparkContext.textFile('data/20191021_policeAddress.txt')
myrdd = _myrdd.map(lambda x:x.split('\n'))
myDf= spark.createDataFrame(myrdd, ['sent'])

In [25]:
myDf.show()

+----------------------------------+
|                              sent|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
|                                  |
| 국민의 안전을 위해 밤낮없이 애...|
|                                  |
|오늘 홍조근정훈장을 받으신 중앙...|
|                                  |
|           사랑하는 경찰관 여러분,|
|                                  |
|여러분의 헌신적 노력으로 우리의...|
|                                  |
| 치안의 개선은 국민의 체감으로 ...|
|                                  |
| 한국을 찾는 외국 관광객들도 우...|
|                                  |
|   올해는 ‘경찰의 날’에 맞춰 국...|
|                                  |
|         자랑스러운 경찰관 여러분,|
|                                  |
| 경찰헌장은 “나라와 겨레를 위하...|
|                                  |
+----------------------------------+
only showing top 20 rows



* 2) 단어로 분리해서, 출력

In [53]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)

In [54]:
tokDf.show()

+----------------------------------+--------------------------------+
|                              sent|                           words|
+----------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|
|                                  |                              []|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|
|                                  |                              []|
| 치안의 개선은 국민의 체감으로 ...|  [치안의, 개선은, 국민의, 체...|
|                                  |                              []|
| 한국을 찾는 외국 관광객들도 우...|  [한국을, 찾는, 외국, 관광객...|
|                                  |     

* 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [55]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")

In [56]:
stopwords = [u"돌", u"참", u"더", u"될", u"그", u"등", u"법", u"큰", u"수"]
for e in stopwords:
    print(e, end=' ')

돌 참 더 될 그 등 법 큰 수 

* 4) 불용어 제거하고, 출력

In [57]:
stop.setStopWords(stopwords)
stopDf=stop.transform(tokDf)

In [58]:
stopDf.show()

+----------------------------------+--------------------------------+--------------------------------+
|                              sent|                           words|                         nostops|
+----------------------------------+--------------------------------+--------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|
|                                  |                              []|                              []|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|
|                                  |                              []|                              []|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|
|                                  |                              []|                              []|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|     [사랑하는, 경찰관, 여러분,]|
|                                  |                              []|                              []

* 5) TF-IDF를 계산하고, 출력

In [59]:
from pyspark.ml.feature import HashingTF, IDF
hashTF = HashingTF(inputCol = "nostops", outputCol="hash")

In [61]:
hashDf = hashTF.transform(stopDf)

In [62]:
hashDf.show()

+----------------------------------+--------------------------------+--------------------------------+--------------------+
|                              sent|                           words|                         nostops|                hash|
+----------------------------------+--------------------------------+--------------------------------+--------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|(262144,[162,8073...|
|                                  |                              []|                              []|(262144,[249180],...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|(262144,[5341,630...|
|                                  |                              []|                              []|(262144,[249180],...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 홍조근정훈장을, 받으신...|(262144,[29823,32...|
|                                  |                              []|                              []|(262144

In [63]:
idf = IDF(inputCol="hash", outputCol="idf")

In [65]:
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)

In [66]:
idfDf.select("sent","words","idf").show()

+----------------------------------+--------------------------------+--------------------+
|                              sent|                           words|                 idf|
+----------------------------------+--------------------------------+--------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|(262144,[162,8073...|
|                                  |                              []|(262144,[249180],...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|(262144,[5341,630...|
|                                  |                              []|(262144,[249180],...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|(262144,[29823,32...|
|                                  |                              []|(262144,[249180],...|
|           사랑하는 경찰관 여러분,|     [사랑하는, 경찰관, 여러분,]|(262144,[118171,1...|
|                                  |                              []|(262144,[249180],...|
|여러분의 헌신적 노력으로 우리의...| [여러분의, 헌신적, 노력으로,...|(262144,[11202,28...|
|                      

* 6) TF-IDF 컬럼을 features로 구성, 출력

In [67]:
reDf = idfDf.withColumnRenamed('idf','features')
reDf.show()

+----------------------------------+--------------------------------+--------------------------------+--------------------+--------------------+
|                              sent|                           words|                         nostops|                hash|            features|
+----------------------------------+--------------------------------+--------------------------------+--------------------+--------------------+
|  존경하는 국민 여러분, 경찰관 ...|   [존경하는, 국민, 여러분,, ...|   [존경하는, 국민, 여러분,, ...|(262144,[162,8073...|(262144,[162,8073...|
|                                  |                              []|                              []|(262144,[249180],...|(262144,[249180],...|
| 국민의 안전을 위해 밤낮없이 애...|  [국민의, 안전을, 위해, 밤낮...|  [국민의, 안전을, 위해, 밤낮...|(262144,[5341,630...|(262144,[5341,630...|
|                                  |                              []|                              []|(262144,[249180],...|(262144,[249180],...|
|오늘 홍조근정훈장을 받으신 중앙...|[오늘, 홍조근정훈장을, 받으신...|[오늘, 